In [1]:
import pandas as pd

# Đọc file Excel
data = pd.read_excel("data.xlsx")
data.rename(columns={"SR4": "ST4"}, inplace=True)

# Xem 5 dòng đầu
data.head()

# Kiểm tra thông tin cơ bản
data.info()

# Kiểm tra giá trị thiếu
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gender  199 non-null    object
 1   Age     199 non-null    object
 2   Income  199 non-null    object
 3   Edu     199 non-null    object
 4   TOUI    199 non-null    int64 
 5   ST1     199 non-null    int64 
 6   ST2     199 non-null    int64 
 7   ST3     199 non-null    int64 
 8   ST4     199 non-null    int64 
 9   SQ1     199 non-null    int64 
 10  SQ2     199 non-null    int64 
 11  SQ3     199 non-null    int64 
 12  SQ4     199 non-null    int64 
 13  SQ5     199 non-null    int64 
 14  FOMO1   199 non-null    int64 
 15  FOMO2   199 non-null    int64 
 16  FOMO3   199 non-null    int64 
 17  FOMO4   199 non-null    int64 
 18  FOMO5   199 non-null    int64 
 19  PU1     199 non-null    int64 
 20  PU2     199 non-null    int64 
 21  PU3     199 non-null    int64 
 22  PU4     199 non-null    in

Gender    0
Age       0
Income    0
Edu       0
TOUI      0
ST1       0
ST2       0
ST3       0
ST4       0
SQ1       0
SQ2       0
SQ3       0
SQ4       0
SQ5       0
FOMO1     0
FOMO2     0
FOMO3     0
FOMO4     0
FOMO5     0
PU1       0
PU2       0
PU3       0
PU4       0
PU5       0
BE1       0
BE2       0
BE3       0
BE4       0
BE5       0
SE1       0
SE2       0
SE3       0
SE4       0
US1       0
US2       0
US3       0
US4       0
US5       0
dtype: int64

In [2]:
!pip install openpyxl



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


LÀM SẠCH DỮ LIỆU

In [3]:
data.info()
data.describe()
data.isnull().sum()
data.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gender  199 non-null    object
 1   Age     199 non-null    object
 2   Income  199 non-null    object
 3   Edu     199 non-null    object
 4   TOUI    199 non-null    int64 
 5   ST1     199 non-null    int64 
 6   ST2     199 non-null    int64 
 7   ST3     199 non-null    int64 
 8   ST4     199 non-null    int64 
 9   SQ1     199 non-null    int64 
 10  SQ2     199 non-null    int64 
 11  SQ3     199 non-null    int64 
 12  SQ4     199 non-null    int64 
 13  SQ5     199 non-null    int64 
 14  FOMO1   199 non-null    int64 
 15  FOMO2   199 non-null    int64 
 16  FOMO3   199 non-null    int64 
 17  FOMO4   199 non-null    int64 
 18  FOMO5   199 non-null    int64 
 19  PU1     199 non-null    int64 
 20  PU2     199 non-null    int64 
 21  PU3     199 non-null    int64 
 22  PU4     199 non-null    in

np.int64(15)

KIỂM ĐỊNH ĐỘ TIN CẬY THANG ĐO (Cronbach’s α)

In [4]:
# Bước 2: Cronbach's Alpha và phân tích item
import pandas as pd
import numpy as np

# giả sử data đã nạp và tên là `data`
# xác nhận lại các nhóm biến (đã chỉnh ST4)
scarcity_cols = ['ST1', 'ST2', 'ST3', 'ST4','SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5']
scarcity_time_cols = ['ST1', 'ST2', 'ST3', 'ST4']
scarcity_quantity_cols = ['SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5']
fomo_cols     = ['FOMO1', 'FOMO2', 'FOMO3', 'FOMO4', 'FOMO5']
pu_cols       = ['PU1', 'PU2', 'PU3', 'PU4', 'PU5']
be_cols       = ['BE1', 'BE2', 'BE3', 'BE4', 'BE5']
se_cols       = ['SE1', 'SE2', 'SE3', 'SE4']
us_cols       = ['US1', 'US2', 'US3', 'US4', 'US5']

groups = {
    "Scarcity": scarcity_cols,
    "FOMO": fomo_cols,
    "PU": pu_cols,
    "BE": be_cols,
    "SE": se_cols,
    "US": us_cols
}

def cronbach_alpha(df_items):
    """Cronbach's alpha bằng công thức: alpha = (N/(N-1))*(1 - sum(var_i)/var_total)"""
    df = df_items.dropna(axis=0, how='any')  # loại hàng có missing nếu có
    item_vars = df.var(axis=0, ddof=1)
    total_var = df.sum(axis=1).var(ddof=1)
    N = df.shape[1]
    if N <= 1:
        return np.nan
    alpha = (N / (N - 1)) * (1 - item_vars.sum() / total_var)
    return alpha

def item_total_correlations(df_items):
    """Tính hệ số tương quan giữa từng item và tổng của các item còn lại (item-total)"""
    res = {}
    for col in df_items.columns:
        others = df_items.drop(columns=[col]).sum(axis=1)
        r = df_items[col].corr(others)
        res[col] = r
    return res

def alpha_if_item_deleted(df_items):
    """Tính α nếu loại từng item"""
    res = {}
    for col in df_items.columns:
        df2 = df_items.drop(columns=[col])
        res[col] = cronbach_alpha(df2)
    return res

# Chạy cho từng nhóm và in kết quả
for name, cols in groups.items():
    print("=== Nhóm:", name, " (", len(cols), "items ) ===")
    # kiểm tra tồn tại các cột
    missing_cols = [c for c in cols if c not in data.columns]
    if missing_cols:
        print("  Lỗi: thiếu cột trong dữ liệu:", missing_cols)
        print()
        continue

    df_grp = data[cols].astype(float)  # đảm bảo kiểu số
    alpha = cronbach_alpha(df_grp)
    print(f"  Cronbach's alpha = {alpha:.3f}")

    # item-total correlations
    itc = item_total_correlations(df_grp)
    print("  Item-total correlations:")
    for k, v in itc.items():
        print(f"    {k}: {v:.3f}")

    # alpha if item deleted
    a_del = alpha_if_item_deleted(df_grp)
    print("  Alpha if item deleted:")
    for k, v in a_del.items():
        print(f"    drop {k}: alpha = {v:.3f}")
    print()
#Kết luận Bước 2: Tất cả α > 0.9 → rất tốt, thang đo đáng tin cậy, không cần loại mục nào. Không có item nào làm tăng alpha nếu bị loại → giữ nguyên toàn bộ 38 câu hỏi.

=== Nhóm: Scarcity  ( 9 items ) ===
  Cronbach's alpha = 0.945
  Item-total correlations:
    ST1: 0.719
    ST2: 0.770
    ST3: 0.793
    ST4: 0.783
    SQ1: 0.785
    SQ2: 0.787
    SQ3: 0.786
    SQ4: 0.823
    SQ5: 0.816
  Alpha if item deleted:
    drop ST1: alpha = 0.942
    drop ST2: alpha = 0.939
    drop ST3: alpha = 0.938
    drop ST4: alpha = 0.939
    drop SQ1: alpha = 0.938
    drop SQ2: alpha = 0.938
    drop SQ3: alpha = 0.938
    drop SQ4: alpha = 0.936
    drop SQ5: alpha = 0.937

=== Nhóm: FOMO  ( 5 items ) ===
  Cronbach's alpha = 0.919
  Item-total correlations:
    FOMO1: 0.815
    FOMO2: 0.814
    FOMO3: 0.850
    FOMO4: 0.718
    FOMO5: 0.763
  Alpha if item deleted:
    drop FOMO1: alpha = 0.896
    drop FOMO2: alpha = 0.896
    drop FOMO3: alpha = 0.888
    drop FOMO4: alpha = 0.915
    drop FOMO5: alpha = 0.907

=== Nhóm: PU  ( 5 items ) ===
  Cronbach's alpha = 0.943
  Item-total correlations:
    PU1: 0.838
    PU2: 0.876
    PU3: 0.833
    PU4: 0.870
    PU

In [5]:
from numpy.linalg import eigvals

def composite_reliability_and_ave(df_items):
    """
    Tính Composite Reliability (CR) và Average Variance Extracted (AVE)
    Dựa trên ma trận tương quan và eigenvalues (theo hướng tiếp cận CFA).
    """
    df = df_items.dropna()
    corr_matrix = df.corr()

    # Trích eigenvalues để ước lượng phương sai chung
    eig_vals = eigvals(corr_matrix)
    first_eig = np.real(eig_vals[0])

    # Hệ số tải giả định (factor loading) ~ căn bậc hai của tương quan với nhân tố chung
    loadings = np.sqrt(np.clip(df.corrwith(df.sum(axis=1)), 0, 1))

    # Bình phương hệ số tải
    load_sq = loadings ** 2

    # Composite Reliability (CR) = (Σλ)² / [(Σλ)² + Σ(1−λ²)]
    cr = (loadings.sum())**2 / ((loadings.sum())**2 + ((1 - load_sq).sum()))

    # Average Variance Extracted (AVE) = Σ(λ²) / n
    ave = load_sq.mean()

    return cr, ave


# Bổ sung tính CR và AVE cho từng nhóm
for name, cols in groups.items():
    print("=== Nhóm:", name, "===")
    df_grp = data[cols].astype(float).dropna()

    alpha = cronbach_alpha(df_grp)
    cr, ave = composite_reliability_and_ave(df_grp)

    print(f"  Cronbach's Alpha (CA): {alpha:.3f}")
    print(f"  Composite Reliability (CR): {cr:.3f}")
    print(f"  Average Variance Extracted (AVE): {ave:.3f}")
    print()


=== Nhóm: Scarcity ===
  Cronbach's Alpha (CA): 0.945
  Composite Reliability (CR): 0.978
  Average Variance Extracted (AVE): 0.833

=== Nhóm: FOMO ===
  Cronbach's Alpha (CA): 0.919
  Composite Reliability (CR): 0.971
  Average Variance Extracted (AVE): 0.870

=== Nhóm: PU ===
  Cronbach's Alpha (CA): 0.943
  Composite Reliability (CR): 0.979
  Average Variance Extracted (AVE): 0.902

=== Nhóm: BE ===
  Cronbach's Alpha (CA): 0.928
  Composite Reliability (CR): 0.974
  Average Variance Extracted (AVE): 0.882

=== Nhóm: SE ===
  Cronbach's Alpha (CA): 0.905
  Composite Reliability (CR): 0.968
  Average Variance Extracted (AVE): 0.883

=== Nhóm: US ===
  Cronbach's Alpha (CA): 0.901
  Composite Reliability (CR): 0.966
  Average Variance Extracted (AVE): 0.849



In [6]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ví dụ: chọn các biến độc lập (thay bằng tên biến của Na)
X = data[['Scarcity', 'FOMO', 'PU', 'BE', 'SE', 'US']]
X = sm.add_constant(X)  # thêm cột hằng số

# Tính VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                   for i in range(X.shape[1])]

print(vif_data)


KeyError: "None of [Index(['Scarcity', 'FOMO', 'PU', 'BE', 'SE', 'US'], dtype='object')] are in the [columns]"

TẠO BIẾN TỔNG HỢP, PHÂN TÍCH MÔ TẢ

In [ ]:
# Bước 3: Tạo biến tổng hợp
data['Scarcity'] = data[['ST1','ST2','ST3','ST4','SQ1','SQ2','SQ3','SQ4','SQ5']].mean(axis=1)
data['scarcity_time'] = data[['ST1','ST2','ST3','ST4']].mean(axis=1)
data['scarcity_quantity'] = data[['SQ1','SQ2','SQ3','SQ4','SQ5']].mean(axis=1)
data['FOMO'] = data[['FOMO1','FOMO2','FOMO3','FOMO4','FOMO5']].mean(axis=1)
data['PU'] = data[['PU1','PU2','PU3','PU4','PU5']].mean(axis=1)
data['BE'] = data[['BE1','BE2','BE3','BE4','BE5']].mean(axis=1)
data['SE'] = data[['SE1','SE2','SE3','SE4']].mean(axis=1)
data['US'] = data[['US1','US2','US3','US4','US5']].mean(axis=1)

# Thống kê mô tả
desc = data[['Scarcity','FOMO','PU','BE','SE','US']].describe().T
print(desc)

# (Tuỳ chọn) Vẽ boxplot để xem phân bố
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.boxplot(data=data[['Scarcity','FOMO','PU','BE','SE','US']])
plt.title("Distribution of Mean Scores Across Scales", fontsize=13)
plt.ylabel("Mean Value")
plt.savefig('Phân bố điểm trung bình các thang đo')
plt.show()
#Tất cả các thang đo đạt độ tin cậy cao (Cronbach’s alpha > 0.9) và dữ liệu mô tả ổn định → Không cần loại biến hay xử lý thêm.

PHÂN TÍCH NHÂN TỐ KHÁM PHÁ (EFA)

In [ ]:
!pip install factor_analyzer


In [ ]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

In [ ]:
cols = ['ST1','ST2','ST3','ST4','SQ1','SQ2','SQ3','SQ4','SQ5',
        'FOMO1','FOMO2','FOMO3','FOMO4','FOMO5',
        'PU1','PU2','PU3','PU4','PU5',
        'BE1','BE2','BE3','BE4','BE5',
        'SE1','SE2','SE3','SE4',
        'US1','US2','US3','US4','US5']
data_factors = data[cols]

In [ ]:
# Kiểm tra độ phù hợp của dữ liệu cho EFA
kmo_all, kmo_model = calculate_kmo(data_factors)
chi_square_value, p_value = calculate_bartlett_sphericity(data_factors)

print(f"KMO overall: {kmo_model}")
print(f"Bartlett’s Test p-value: {p_value}")
#đọc kết quả: KMO > 0.6 → dữ liệu phù hợp để làm EFA. Bartlett’s Test p < 0.05 → các biến có tương quan, nên chạy EFA được.

In [ ]:
# Kiểm tra eigenvalues để xác định số nhân tố nên giữ lại
fa = FactorAnalyzer(rotation=None)
fa.fit(data_factors)
ev, v = fa.get_eigenvalues()

# Hiển thị eigenvalues
import matplotlib.pyplot as plt

plt.scatter(range(1, data_factors.shape[1]+1), ev)
plt.plot(range(1, data_factors.shape[1]+1), ev)
plt.title('Scree Plot')
plt.xlabel('Factor')
plt.ylabel('Eigenvalue')
plt.grid()
plt.savefig('biểu đồ scree plot')
plt.show()

Khi EFA xác định được 6 nhân tố, điều đó nghĩa là:
Tập hợp các biến quan sát (câu hỏi trong bảng khảo sát) được gộp lại thành 6 nhóm chính.
Các nhân tố khác (từ nhân tố thứ 7 trở đi) có Eigenvalue < 1, nên không đủ điều kiện để được giữ lại (vì chúng giải thích rất ít phương sai, < 1 biến gốc).
Nói cách khác, mô hình ban đầu có thể có 6 thang đo lý thuyết, và EFA xác nhận đúng 6 nhóm là phù hợp — các nhóm khác (nếu có) chỉ là nhiễu, không có ý nghĩa.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
factors = data[['Scarcity', 'FOMO', 'PU', 'BE', 'SE', 'US']]

# Tính ma trận tương quan Pearson
corr_matrix = factors.corr(method='pearson')
print(corr_matrix)

# Vẽ biểu đồ heatmap để trực quan hoá
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("Pearson Correlation Matrix")
plt.savefig("Ma trận tương quan Pearson giữa các nhân tố")
plt.show()


PHÂN TÍCH HỒI QUY ĐA BIẾN

In [ ]:
!pip install statsmodels


In [ ]:
import statsmodels.api as sm

# Xác định biến độc lập và phụ thuộc
X = data[['Scarcity', 'FOMO', 'PU', 'BE', 'SE']]
y = data['US']

# Thêm hằng số vào mô hình
X = sm.add_constant(X)

# Chạy hồi quy tuyến tính đa biến
model = sm.OLS(y, X).fit()

# In kết quả
print(model.summary())
print ('Durbin–Watson = 1.921 → Không có tự tương quan phần dư (vì DW nằm trong khoảng 1.5–2.5).Kiểm định Jarque–Bera có p = 0.0009 < 0.05 → Phần dư hơi lệch chuẩn nhẹ, nhưng mức độ chấp nhận được. Cond. No. = 42.5 (< 100) → Không có vấn đề đa cộng tuyến nghiêm trọng.')

In [ ]:
!pip install pingouin


TÁC ĐỘNG GIÁN TIẾP

In [ ]:
import pandas as pd
from statsmodels.stats.mediation import Mediation
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
model_direct = smf.ols("US ~ Scarcity + FOMO", data=data).fit()
print(model_direct.summary())

In [ ]:
model_direct = smf.ols("US ~ Scarcity + PU", data=data).fit()
print(model_direct.summary())

In [ ]:
import pingouin as pg
import pandas as pd

# Mediation Scarcity -> FOMO -> US
med_fomo = pg.mediation_analysis(data=data, x='Scarcity', m='FOMO', y='US', 
                                alpha=0.05, n_boot=5000)
print("=== Mediation: Scarcity -> FOMO -> US ===")
print(med_fomo)

# Mediation Scarcity -> PU -> US
med_pu = pg.mediation_analysis(data=data, x='Scarcity', m='PU', y='US',
                               alpha=0.05, n_boot=5000)
print("\n=== Mediation: Scarcity -> PU -> US ===")
print(med_pu)


TÁC ĐỘNG TRỰC TIẾP

In [ ]:
#Scarcity ảnh hưởng trực tiếp đến US
model_direct = smf.ols("US ~ Scarcity", data=data).fit()
print(model_direct.summary())

In [ ]:
#Scarcity, bE ảnh hưởng trực tiếp đến US
model_direct = smf.ols("US ~ Scarcity + BE", data=data).fit()
print(model_direct.summary())

In [ ]:
#Scarcity, SE ảnh hưởng trực tiếp đến US
model_direct = smf.ols("US ~ Scarcity + SE", data=data).fit()
print(model_direct.summary())

In [ ]:
# PHẦN BỔ SUNG: THỐNG KÊ MÔ TẢ BIẾN NHÂN KHẨU HỌC
# ==============================================================================
import seaborn as sns
import matplotlib.pyplot as plt

print("\n\n--- THỐNG KÊ MÔ TẢ BIẾN NHÂN KHẨU HỌC ---")

# Danh sách các biến nhân khẩu học dạng phân loại
demographic_categorical_cols = ['Gender', 'Age', 'Income', 'Edu']

# Lặp qua từng biến để tính tần suất và vẽ biểu đồ
for col in demographic_categorical_cols:
    print(f"\n--- Phân tích biến: {col} ---")
    
    # In ra bảng tần suất và tỷ lệ
    frequency_table = pd.DataFrame({
        'Tần suất (Count)': data[col].value_counts(),
        'Tỷ lệ (%)': data[col].value_counts(normalize=True).mul(100).round(2)
    })
    print(frequency_table)
    
    # Vẽ biểu đồ cột
    plt.figure(figsize=(10, 6))
    sns.countplot(y=data[col], order=data[col].value_counts().index, palette='viridis')
    plt.title(f'Biểu đồ phân bố của biến {col}', fontsize=14)
    plt.xlabel('Số lượng (Count)', fontsize=12)
    plt.ylabel(col, fontsize=12)
    plt.tight_layout()
    plt.show()

# Thống kê mô tả cho biến định lượng TOUI (Time On Using Internet)
print("\n--- Phân tích biến: TOUI ---")
print(data['TOUI'].describe())

# Vẽ biểu đồ histogram để xem phân bố của TOUI
plt.figure(figsize=(10, 6))
sns.histplot(data['TOUI'], kde=True, bins=15)
plt.title('Biểu đồ phân bố của biến TOUI', fontsize=14)
plt.xlabel('Giá trị', fontsize=12)
plt.ylabel('Tần suất', fontsize=12)
plt.show()

In [ ]:
# ==============================================================================
# PHẦN BỔ SUNG: TÍCH HỢP BIẾN NHÂN KHẨU HỌC VÀO MÔ HÌNH HỒI QUY (PHIÊN BẢN SỬA LỖI)
# ==============================================================================
import statsmodels.api as sm
import pandas as pd

print("\n\n--- PHÂN TÍCH HỒI QUY MỚI (BAO GỒM BIẾN NHÂN KHẨU HỌC) ---")

# Bước 1: Mã hóa các biến phân loại thành các biến giả (dummy variables)
# Thêm dtype=float để đảm bảo các cột mới được tạo ra là số
data_encoded = pd.get_dummies(data, columns=['Gender', 'Age', 'Income', 'Edu'], drop_first=True, dtype=float)

# Bước 2: Xác định các biến độc lập (X) và biến phụ thuộc (y)
original_predictors = ['Scarcity', 'FOMO', 'PU', 'BE', 'SE']
demographic_predictors = [col for col in data_encoded.columns if any(prefix in col for prefix in ['Gender_', 'Age_', 'Income_', 'Edu_'])]

# Ép kiểu cho biến TOUI để đảm bảo nó là số, chuyển lỗi thành NaN
data_encoded['TOUI'] = pd.to_numeric(data_encoded['TOUI'], errors='coerce')

# Kết hợp tất cả các biến sẽ sử dụng trong mô hình
X_full = data_encoded[original_predictors + demographic_predictors + ['TOUI']]
y_full = data_encoded['US']

# *** BƯỚC SỬA LỖI QUAN TRỌNG ***
# 1. Gộp X và y lại để xử lý NaN đồng bộ
model_data = pd.concat([y_full, X_full], axis=1)

# 2. In ra số dòng TRƯỚC khi loại bỏ giá trị thiếu
print(f"Số lượng quan sát ban đầu: {len(model_data)}")

# 3. Xóa tất cả các hàng có chứa bất kỳ giá trị thiếu (NaN) nào
#    Điều này đảm bảo mô hình sẽ không gặp lỗi và y, X luôn khớp nhau
model_data.dropna(inplace=True)

print(f"Số lượng quan sát hợp lệ (sau khi xóa NaN): {len(model_data)}")

# 4. Tách lại y và X sau khi đã làm sạch
y_clean = model_data['US']
X_clean = model_data.drop('US', axis=1)

# Kiểm tra lần cuối để chắc chắn tất cả các cột đã là số
if (X_clean.dtypes == 'object').any():
    print("\nCẢNH BÁO: Vẫn còn cột 'object' trong dữ liệu. Vui lòng kiểm tra lại dữ liệu nguồn.")
    print(X_clean.dtypes[X_clean.dtypes == 'object'])
else:
    print("\n✔️ Kiểm tra thành công: Tất cả các cột đầu vào cho mô hình đã là số.")

    # Bước 3: Thêm hằng số (intercept) vào mô hình
    X_final = sm.add_constant(X_clean)

    # Bước 4: Chạy mô hình hồi quy OLS với dữ liệu đã được làm sạch hoàn toàn
    model_full = sm.OLS(y_clean, X_final).fit()

    # Bước 5: In kết quả chi tiết của mô hình mới
    print(model_full.summary())

In [ ]:
# Mã hóa biến nhân khẩu học
data['Gender'] = data['Gender'].map({'Nam': 1, 'Nữ': 0})
data['Age_cat'] = data['Age'].astype('category').cat.codes
data['Income_cat'] = data['Income'].astype('category').cat.codes
data['Edu_cat'] = data['Edu'].astype('category').cat.codes


In [ ]:
import statsmodels.api as sm

for gender, subset in data.groupby('Gender'):
    name = 'Nam' if gender == 1 else 'Nữ'
    X = subset[['Scarcity', 'FOMO', 'PU', 'BE', 'SE']]
    y = subset['US']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    print(f"\n=== Kết quả hồi quy nhóm {name} ===")
    print(model.summary())

In [ ]:
for cat, subset in data.groupby('Age_cat'):
    print(f"\n=== Nhóm tuổi: {subset['Age'].iloc[0]} ===")
    X = subset[['Scarcity', 'FOMO', 'PU', 'BE', 'SE']]
    y = subset['US']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    print(model.summary())


In [ ]:
def run_group_regression(df, group_col):
    results = []
    for group, data in df.groupby(group_col):
        model = smf.ols('US ~ Scarcity + FOMO + PU + BE + SE', data=data).fit()
        results.append({
            'Nhóm': group,
            'R²': round(model.rsquared, 3),
            'Adj R²': round(model.rsquared_adj, 3),
            'F_p': round(model.f_pvalue, 4),
            'PU_coef': round(model.params['PU'], 3),
            'PU_p': round(model.pvalues['PU'], 4),
            'FOMO_coef': round(model.params['FOMO'], 3),
            'FOMO_p': round(model.pvalues['FOMO'], 4),
            'Scarcity_coef': round(model.params['Scarcity'], 3),
            'Scarcity_p': round(model.pvalues['Scarcity'], 4),
            'SE_coef': round(model.params['SE'], 3),
            'SE_p': round(model.pvalues['SE'], 4),
            'BE_coef': round(model.params['BE'], 3),
            'BE_p': round(model.pvalues['BE'], 4),
        })
    return pd.DataFrame(results)


In [ ]:
# So sánh theo giới tính
gender_results = run_group_regression(data, 'Gender')

# So sánh theo độ tuổi
age_results = run_group_regression(data, 'Age')

# So sánh theo thu nhập
income_results = run_group_regression(data, 'Income')
print("=== So sánh theo giới tính ===")
print(gender_results)

print("\n=== So sánh theo độ tuổi ===")
print(age_results)

print("\n=== So sánh theo thu nhập ===")
print(income_results)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Nếu chưa có nhóm scarcity thì chạy dòng này
if 'Scarcity_Group' not in data.columns:
    data['Scarcity_Group'] = pd.qcut(data['Scarcity'], q=3, labels=['Low', 'Medium', 'High'])

# Tùy chọn 1: Biểu đồ tán xạ FOMO vs US (chia theo nhóm Scarcity)
plt.figure(figsize=(8,6))
sns.scatterplot(data=data, x='FOMO', y='US', hue='Scarcity_Group', palette='coolwarm', alpha=0.7)
sns.regplot(data=data, x='FOMO', y='US', scatter=False, color='black', ci=None)
plt.title('Relationship between FOMO and Impulsive Buying Intention (US)\nGrouped by the Level of Perceived Scarcity')
plt.xlabel('Level of FOMO')
plt.ylabel('Impulsive Buying Intention (US)')
plt.legend(title='Scarcity Level')
plt.savefig('fomo đến us group')
plt.show()

# Tùy chọn 2: Biểu đồ tán xạ PU vs US (chia theo nhóm Scarcity)
plt.figure(figsize=(8,6))
sns.scatterplot(data=data, x='PU', y='US', hue='Scarcity_Group', palette='coolwarm', alpha=0.7)
sns.regplot(data=data, x='PU', y='US', scatter=False, color='black', ci=None)
plt.title('Relationship between Perceived Urgency (PU) and Impulsive Buying Intention (US)\nGrouped by the Level of Perceived Scarcity')
plt.xlabel('Level of PU')
plt.ylabel('Impulsive Buying Intention (US)')
plt.legend(title='Scarcity Level')
plt.savefig('pu đến us group')
plt.show()




In [ ]:
sns.pairplot(
    data,
    vars=["FOMO", "PU", "US", "Scarcity"],
    hue="Scarcity_Group",
    palette="husl",
    diag_kind="kde"
)
plt.suptitle("Pairwise Relationships between FOMO, PU, Scarcity, and Impulsive Buying", y=1.02, fontsize=13, fontweight='bold')
plt.savefig('pairwise')
plt.show()


In [ ]:
# ---------------------------
# 2. Scatter + regression: FOMO vs US, colored by Scarcity, facet by Gender
g = sns.lmplot(
    data=data,
    x='FOMO', y='US',
    hue='Scarcity_Group',
    col='Gender',
    palette=['#4A90E2','#BDBDBD','#FF7043'],
    height=4, aspect=1,
    scatter_kws={'alpha':0.6, 's':50},
    markers=['o','s','D'],
    ci=95
)
g.fig.suptitle("FOMO vs Impulsive Buying (US) by Scarcity Level and Gender", fontsize=14, y=1.02)
plt.show()  # ✅ thêm dòng này
plt.savefig("scatter_FOMO_US_by_Scarcity_Gender.png")


In [ ]:
# ---------------------------
# 2. Scatter + regression: FOMO vs US, colored by Scarcity, facet by Gender
g = sns.lmplot(
    data=data,
    x='PU', y='US',
    hue='Scarcity_Group',
    col='Gender',
    palette=['#4A90E2','#BDBDBD','#FF7043'],
    height=4, aspect=1,
    scatter_kws={'alpha':0.6, 's':50},
    markers=['o','s','D'],
    ci=95
)
g.fig.suptitle("PU vs Impulsive Buying (US) by Scarcity Level and Gender", fontsize=14, y=1.02)
plt.show()  # ✅ thêm dòng này
plt.savefig("scatter_PU_US_by_Scarcity_Gender.png")


In [ ]:
# Import thư viện
import pandas as pd

# Giả sử file của bạn đã load thành DataFrame 'data'
# data = pd.read_excel("data.xlsx")

# Thống kê tần suất và tỉ lệ phần trăm cho các biến nhân khẩu học
demo_vars = ['Gender', 'Age', 'Edu', 'Income', 'TOUI']

for var in demo_vars:
    print(f"\n--- {var.upper()} ---")
    freq_table = data[var].value_counts(dropna=False)
    percent_table = round(data[var].value_counts(normalize=True, dropna=False) * 100, 2)
    summary_df = pd.DataFrame({'Count': freq_table, 'Percent (%)': percent_table})
    print(summary_df)


In [ ]:
summary_tables = {}
for var in demo_vars:
    freq = data[var].value_counts(dropna=False)
    pct = data[var].value_counts(normalize=True, dropna=False)*100
    summary_tables[var] = pd.DataFrame({'Count': freq, 'Percent': pct.round(2)})

# Hiển thị tóm tắt tất cả trong 1 bảng gọn
demo_summary = pd.concat(summary_tables, axis=0)
demo_summary


In [ ]:
!pip install graphviz statsmodels pingouin
# On some systems you also need the system graphviz:
# apt-get install graphviz      # (Linux)
# brew install graphviz         # (mac)


In [ ]:
import matplotlib.pyplot as plt

# Tạo khung hình
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_xlim(0, 12)
ax.set_ylim(0, 6)
ax.axis("off")

# ======= Vẽ các ô (biến trong mô hình) =======
ax.text(1, 3, "Scarcity\n(X)", ha="center", va="center", fontsize=11,
        bbox=dict(boxstyle="round,pad=0.4", fc="#cce5ff", ec="#003366"))

ax.text(5, 3, "FOMO\n(Mediator 1)", ha="center", va="center", fontsize=11,
        bbox=dict(boxstyle="round,pad=0.4", fc="#d4edda", ec="#155724"))

ax.text(9, 3, "Purchase Intention\n(Y)", ha="center", va="center", fontsize=11,
        bbox=dict(boxstyle="round,pad=0.4", fc="#f8d7da", ec="#721c24"))

# ======= Vẽ thêm các biến kiểm soát hoặc ảnh hưởng phụ (nếu có) =======
ax.text(5, 1, "Perceived Usefulness\n(Control Variable)", ha="center", va="center", fontsize=10,
        bbox=dict(boxstyle="round,pad=0.3", fc="#fff3cd", ec="#856404"))

# ======= Vẽ các mũi tên =======
# Đường chính (X -> M -> Y)
ax.annotate("", xy=(4, 3), xytext=(2, 3), arrowprops=dict(arrowstyle="->", lw=2))
ax.annotate("", xy=(8, 3), xytext=(6, 3), arrowprops=dict(arrowstyle="->", lw=2))
# Đường trực tiếp X -> Y
ax.annotate("", xy=(8, 3.8), xytext=(2, 3.8), arrowprops=dict(arrowstyle="->", lw=2, linestyle="dashed"))
# Đường phụ PU -> Y
ax.annotate("", xy=(8.5, 2.8), xytext=(5.7, 1.2), arrowprops=dict(arrowstyle="->", lw=1.8, color="gray"))

# ======= Ghi nhãn hệ số =======
ax.text(3, 3.2, "a", fontsize=12)
ax.text(7, 3.2, "b", fontsize=12)
ax.text(5, 4.0, "c’", fontsize=12)
ax.text(7, 2.3, "control", fontsize=10, color="gray")

# ======= Tiêu đề =======
plt.title("Conceptual Framework: OLS Regression and Mediation Model", fontsize=13, fontweight="bold")
plt.show()
